In [20]:
# !pip install --upgrade google-cloud-bigquery-storage

In [21]:
from google.colab import auth
auth.authenticate_user()
from google.cloud import storage, bigquery

project_id = 'wpac-449904'
# Create a storage client
client = bigquery.Client(project=project_id)

In [22]:
query = """
SELECT * FROM `wpac-449904.wpac_data.other_events`
"""
df = client.query(query).to_dataframe()

# Display first few rows
df.head()

,string_field_0,string_field_1,string_field_2,string_field_3,string_field_4
0,NA,searched,Search,Atlanta,WPA_1708
1,NA,searched,Search,Atlanta,WPA_1708
2,NA,searched,Search,Atlanta,WPA_1708
3,NA,searched,Search,Atlanta,WPA_1708
4,NA,searched,Search,Atlanta,WPA_8609


In [23]:
df.isnull().sum()

,0
string_field_0,0
string_field_1,0
string_field_2,0
string_field_3,0
string_field_4,0


In [24]:
headers = ['collectionType', 'eventName', 'eventType', 'partnerRegion', 'WPA_id']

df.columns = headers
print(df.shape)

(576777, 5)


In [25]:
for col in df.columns:
  df = df[df[col] != col]
  print(df[col].value_counts().sort_values())
  print('-'*100)

collectionType
resources      8038
careers       23221
NA            29277
my path      143862
training     160293
jobs         212085
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
eventName
completed a milestone        4625
completed a career goal     13721
sorted results              14470
searched                   118136
completed a task           127513
selected filters           298311
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
eventType
My Path    145859
Search     430917
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
partnerRegion
Remote                  26
NA                      91
Portland               113
Baltimore              376
San Antonio            783
San Diego             1258
West Michigan         1297
Minneapolis          

In [26]:
print(df.shape)

(576776, 5)


In [27]:
dataset_id = "wpac_data"

client = bigquery.Client(project=project_id)
datasets = [dataset.dataset_id for dataset in client.list_datasets()]
print("Available Datasets:", datasets)

if dataset_id not in datasets:
    print(f"Dataset {dataset_id} does not exist. Creating it now...")
    dataset_ref = client.dataset(dataset_id)
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"  # Change if necessary
    client.create_dataset(dataset, exists_ok=True)
    print(f"Dataset {dataset_id} created.")


Available Datasets: ['wpac_data']


In [28]:
import pandas_gbq

dataset_table = "wpac_data.cleaned_other_events"  # Replace with dataset.table name

# Load your CSV into a DataFrame
csv_file_path = "cleaned_other_events.csv"  # Replace with your actual file path
# df = pd.read_csv(csv_file_path)

# Use `pandas_gbq.to_gbq` to upload DataFrame
pandas_gbq.to_gbq(
    df,
    destination_table=dataset_table,
    project_id=project_id,
    if_exists="replace"  # "fail", "replace", "append"
)


100%|██████████| 1/1 [00:00<00:00, 6213.78it/s]
